In [1]:
# Basic imports and path setup
import sys
from pathlib import Path
import pandas as pd

# Ensure project root is on sys.path so we can import the utility module
project_root = Path("..").resolve()  # notebook lives under notebooks/
sys.path.insert(0, str(project_root))

from walkshed_land_value_estimator import (
    load_parcels_from_shapefile,
    Station,
    estimate_walkshed_value,
)

print("Imports OK — notebook ready to run the estimator")

Imports OK — notebook ready to run the estimator


In [2]:
# User-config: update these paths/values before running
# Path to your parcels file (shapefile / geopackage). Example: '../data/mappluto_21v1.shp'
parcels_path = Path(
    "../data/mappluto.shp"
)  # <- set this to your MapPLUTO or parcels file

# Stations CSV: a CSV with columns 'name,lon,lat' describing IBX stations.
# Put your stations file at '../data/stations_ibx.csv' or edit the path below.
stations_csv = Path("../data/stations_ibx.csv")

# Name of the value field in the parcels dataset (update to match your file).
value_field = "full_market_value"  # replace if your dataset uses a different column

# Buffer radius in miles
radius_miles = 0.5

# Adjustment factor for underassessment (1.0 = no adjustment)
adjustment_factor = 1.0

print("Configuration set. Edit paths/fields above as needed.")

Configuration set. Edit paths/fields above as needed.


## Load input data
This cell loads the parcels file and the stations CSV. If you don't have a stations CSV, create one with columns `name,lon,lat`.

In [3]:
# Load parcels
if not parcels_path.exists():
    raise FileNotFoundError(f'Parcels file not found: {parcels_path}
Please set `parcels_path` to the MapPLUTO or parcels file.')

parcels = load_parcels_from_shapefile(str(parcels_path), crs_epsg=4326)
print(f'Loaded parcels: {len(parcels)} rows; CRS={parcels.crs}')

# Load stations
if not stations_csv.exists():
    raise FileNotFoundError(f'Stations CSV not found: {stations_csv}
Create a CSV with columns name,lon,lat and set `stations_csv` accordingly.')

stations_df = pd.read_csv(stations_csv)
required_cols = {'name','lon','lat'}
if not required_cols.issubset(set(stations_df.columns)):
    raise KeyError(f'Stations CSV must contain columns: {required_cols}')

stations = [Station(name=r['name'], lon=float(r['lon']), lat=float(r['lat'])) for _, r in stations_df.iterrows()]
print(f'Loaded {len(stations)} stations from {stations_csv}')

SyntaxError: unterminated f-string literal (detected at line 3) (798618814.py, line 3)

## Run estimator
Call `estimate_walkshed_value` to compute baseline and uplift scenarios.

In [ ]:
baseline, uplift_df = estimate_walkshed_value(
    parcels_gdf=parcels,
    stations=stations,
    value_field=value_field,
    radius_miles=radius_miles,
    adjustment_factor=adjustment_factor,
)

print(f'Baseline property value (adjusted): ${baseline:,.2f}')
print('
Uplift scenarios:')
display(uplift_df)

## Notes and troubleshooting
- Geo packages (geopandas, fiona, shapely) may require system libraries on macOS (GDAL/proj).
- If you used `uv` to manage dependencies, ensure the environment is activated in the notebook kernel.
- If parcel loading fails due to CRS or file format, open the file in QGIS to inspect attributes and coordinate system.